# 演習問題1 分類 (Easy)

## 問題

以下の1~4の変数からアヤメの品種を推定するモデルを作成せよ.

1. Sepal Length（がく片の長さ）
2. Sepal Width（がく片の幅）
3. Petal Length（花びらの長さ）
4. Petal Width（花びらの幅）

今回はSetosa→0, Versicolour→1, Virginica→2 と、目的変数はすでに数値化されている.

#### ヒント　
手順は以下の順番でおこなうとよい.

1. 欠損値があるかを確認する.
2. データを見て, どのアルゴリズムを使うかを決める.
3. ハイパーパラメータの最適化を行う.

(2と3は同時平行に行なってもよい.)

### ランダムフォレストの解答例

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
#ランダムフォレストを使ってみる.
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
iris = load_iris()
iris_df = pd.DataFrame(iris.data)
iris_df.columns = iris.feature_names
#今回の演習の目的変数に相当
iris_df["target_names"] = iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


目的変数にどの品種がどれくらいあるかを確認.

In [4]:
iris_df["target_names"].value_counts()

2    50
1    50
0    50
Name: target_names, dtype: int64

データに欠損値があるかを確認.

In [ ]:
iris_df.isnull().sum()

欠損値がないため、アルゴリズムを検討する。とりあえずハイパーパラメータ調節の楽なランダムフォレストを使ってみる。
説明変数が目的変数とどの程度相関しているかを見てみる.

In [ ]:
iris_df.corr() #各変数の相関値を表示

目的変数と説明変数に強い相関が見られたので, 正答率の高いモデルが作成できそう!

In [5]:
#列名が扱いづらいのでリネームする.
iris_df.columns = ["sepal_length", "sepal_width","petal_length","petal_width","target_names"]

まずは, 大域的にハイパーパラメータの候補を羅列し、そのあと範囲を絞って最適なパラメータを見つける。

In [6]:
clf = RandomForestClassifier()#分類器の生成
#今回試すハイパーパラメータ
params = {'n_estimators'  : [10, 20, 30 ],'max_depth': [1,  5, 10],'random_state':[0]}
X_iris = iris_df[["sepal_length", "sepal_width","petal_length","petal_width"]]
y_iris = iris_df["target_names"]
cv_1 = GridSearchCV(clf, params, cv = 10 , n_jobs =-1,scoring= "accuracy")
cv_1.fit(X_iris, y_iris)
print("計算終了")

計算終了


In [11]:
#スコアを確認
print(cv_1.cv_results_["mean_test_score"])
print(cv_1.cv_results_["params"])

[0.96       0.95333333 0.96       0.95333333 0.96       0.94666667
 0.96       0.95333333 0.96       0.94666667 0.96       0.94666667
 0.96       0.95333333 0.96       0.94666667 0.96       0.95333333]
[{'max_depth': 5, 'n_estimators': 10, 'random_state': 0}, {'max_depth': 5, 'n_estimators': 10, 'random_state': 1}, {'max_depth': 5, 'n_estimators': 15, 'random_state': 0}, {'max_depth': 5, 'n_estimators': 15, 'random_state': 1}, {'max_depth': 5, 'n_estimators': 20, 'random_state': 0}, {'max_depth': 5, 'n_estimators': 20, 'random_state': 1}, {'max_depth': 7, 'n_estimators': 10, 'random_state': 0}, {'max_depth': 7, 'n_estimators': 10, 'random_state': 1}, {'max_depth': 7, 'n_estimators': 15, 'random_state': 0}, {'max_depth': 7, 'n_estimators': 15, 'random_state': 1}, {'max_depth': 7, 'n_estimators': 20, 'random_state': 0}, {'max_depth': 7, 'n_estimators': 20, 'random_state': 1}, {'max_depth': 9, 'n_estimators': 10, 'random_state': 0}, {'max_depth': 9, 'n_estimators': 10, 'random_state': 1},

正答率を確認すると, max_depth = 5から10, estimators　= 10から20くらいの値が良さそう !

In [8]:
params = {'n_estimators'  : [10,15,20],'max_depth': [5,7,9],'random_state':[0,1]}
cv_1 = GridSearchCV(clf, params, cv = 10 , n_jobs =-1,scoring= "accuracy")
cv_1.fit(X_iris, y_iris)
print("範囲限定後の計算完了")

範囲限定後の計算完了


In [10]:
cv_1.cv_results_["mean_test_score"]

array([0.96      , 0.95333333, 0.96      , 0.95333333, 0.96      ,
       0.94666667, 0.96      , 0.95333333, 0.96      , 0.94666667,
       0.96      , 0.94666667, 0.96      , 0.95333333, 0.96      ,
       0.94666667, 0.96      , 0.95333333])

最適なハイパーパラメータを表示

In [9]:
cv_1.best_params_

{'max_depth': 5, 'n_estimators': 10, 'random_state': 0}

### SVMの解答例

今回のデータセットは低次元でデータ数が少ないため、こちらの手法を用いるのがよいと思われる。
SVMの設定すべきハイパーパラメータは以下の2つである。

1. コストパラメータCCは、誤分類をどの程度許容するかを決めるパラメータである。
2. の値が小さいほど単純な決定境界となり, 大きいほど複雑な決定境界となる。

In [15]:
from sklearn.svm import SVC

clf2 = SVC()
params2 = {"C":[0.01,0.1, 1,10],"gamma":[0.01,0.1, 1,10],'random_state':[0]}
cv_2 = GridSearchCV(clf2, params2, cv = 10 , n_jobs =-1, scoring= "accuracy")
cv_2.fit(X_iris, y_iris)
print("計算完了")

計算完了


In [16]:
cv_2.cv_results_["mean_test_score"]#SVMの結果を表示.

array([0.90666667, 0.93333333, 0.96      , 0.77333333, 0.90666667,
       0.94666667, 0.96666667, 0.77333333, 0.94      , 0.97333333,
       0.97333333, 0.93333333, 0.98      , 0.98      , 0.95333333,
       0.94      ])

最適なハイパーパラメータを表示

In [17]:
cv_2.best_params_

{'C': 10, 'gamma': 0.01, 'random_state': 0}

学習済みモデルを用いてクラスを予測したい場合には以下の関数

#### cv_1.predict(データ)

を用いればよい。

### まとめ
ランダムフォレストを用いた場合,  Acurracyが96パーセントのモデル, SVMを用いた場合は98パーセントのモデルを作成することができた
今回の問題の場合にはデータの次元数が少なくデータ数も少ないため、SVMを使うとよいモデルが作れた。